In [1]:
import cv2
import numpy as np

x1, x2, x3 = 0, 0, 0
y1, y2, y3 = 0, 0, 0
X1, X2, X3 = 0, 0, 0
Y1, Y2, Y3 = 0, 0, 0

def follow_line(frame):

    global x1, y1, x2, y2, x3, y3, X1, X2, X3, Y1, Y2, Y3, top, bottom, center

    hsv, x1, y1 = black_point(top, x1, y1)
    hsv, x2, y2 = black_point(center, x2, y2)
    hsv, x3, y3 = black_point(bottom, x3, y3)

 #++++++++++++++++++++++  Желтый блок ++++++++++++++++++++++++++++++++++++++++++
   
    #hsv, X1, Y1 = yellow_point (top, X1, Y1)
    hsv, X2, Y2 = yellow_point (center, X2, Y2)
    #hsv, X3, Y3 = yellow_point (bottom, X3, Y3)

    ang = angle(x1, x2, x3, y1, y2, y3)

    print(ang)

        # Вывод изображения
    cv2.imshow('Contours_top', top)
    cv2.imshow('Contours_center', center)
    cv2.imshow('Contours_bottom', bottom)


def black_point(frame_p, x, y):

    # Преобразование изображения в пространство цветов HSV
    hsv = cv2.cvtColor(frame_p, cv2.COLOR_BGR2HSV)

    # Определение диапазона черного цвета в пространстве HSV
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 30])

    # Создание маски для черного цвета
    mask_b = cv2.inRange(hsv, lower_black, upper_black)

    #Обнаружение границ в диапазоне интенсивности черного 30 - 200
    edged_b = cv2.Canny(mask_b, 30, 200) 
    #Выделение контуров черного цвета
    contours_b, hierarchy = cv2.findContours(edged_b,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #Прорисовка контуров
    cv2.drawContours(frame_p, contours_b, -1, (0, 255, 0), 3)

    #Установка точки в центр рамки черной линии
    b_moments = cv2.moments(mask_b, 1)
    dM01 = b_moments['m01']
    dM10 = b_moments['m10']
    dArea = b_moments['m00']
    if dArea > 150:
        x = int(dM10/dArea)
        y = int(dM01/dArea)
        cv2.circle(frame_p, (x,y), 10, (0,0,255), -1)
    return hsv, x, y

def yellow_point (frame_y, X, Y):
    
    # Преобразование изображения в пространство цветов HSV
    hsv = cv2.cvtColor(frame_y, cv2.COLOR_BGR2HSV)
    
     # Определение диапазона желтого цвета в пространстве HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Создание маски для желтого цвета
    mask_y = cv2.inRange(hsv, lower_yellow, upper_yellow) 

    #Обнаружение границ в диапазоне интенсивности черного 30 - 200
    edged_y = cv2.Canny(mask_y, 30, 200) 
    #Выделение контуров черного цвета
    contours_y, hierarchy = cv2.findContours(edged_y,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #Прорисовка контуров
    cv2.drawContours(frame_y, contours_y, -1, (255, 255, 0), 3)

    #Установка точки в центр рамки черной линии
    y_moments = cv2.moments(mask_y, 1)
    dM01 = y_moments['m01']
    dM10 = y_moments['m10']
    dArea = y_moments['m00']
    if dArea > 150:
        X = int(dM10/dArea)
        Y = int(dM01/dArea)
        cv2.circle(frame_y, (X,Y), 10, (0,255,255), -1)
    return hsv, X, Y

def angle(x1, x2, x3, y1, y2, y3):

    point1 = [x1,y1] #A
    point2 = [x2,y2] #B
    point3 = [x3,y3] #C

    if point2[0] == point1[0] or point3[0] == point2[0]:
        angle_deg = 0
        return angle_deg

    # Вычисляем угловые коэффициенты для прямых AB и BC
    k1 = (point2[1] - point1[1]) / (point2[0] - point1[0])
    k2 = (point3[1] - point2[1]) / (point3[0] - point2[0])

    # Вычисляем угол между прямыми
    angle_rad = np.arctan((k2 - k1) / (1 + k1 * k2))
    angle_deg = np.degrees(angle_rad)

    return angle_deg
u


# Захват видео с камеры
cap = cv2.VideoCapture(1)

while True:
    # Чтение кадра
    ret, frame = cap.read()

    height, widht, channel = frame.shape

    height_3 = height//10

    top = frame[:height_3*2,:]
    center = frame[height_3*2:height_3*8,:]
    bottom = frame[height_3*8:,:]

    if not ret:
     print("Ошибка: не удалось получить кадр из видеопотока.")
    else:
    # Вызов функции следования по линии
        follow_line(frame)

    # Определение направления движения Спри несовпадении точек в диапазоне коридора
    #if x2 < X2 - 50 :
    #     print("Влево")
    #elif x2 > X2 + 50:
    #    print("Вправо")
    #else:
    #     print("Прямо")
    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов и закрытие окон
cap.release()
cv2.destroyAllWindows()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-69.75313200777968
-67.68706620848887
-66.56069378428982
-64.15227722668891
-62.323790223285236
-59.41861266570452
-57.77751983461208
-58.53342425054306
-59.26612994327278
-60.50296985417465
-63.97127948570653
-62.49483201518592
-59.34116610701096
-58.20286657215712
-59.022838398900454
-60.03790364622002
-60.97554798812709
-59.75518596919327
-60.71883341218942
-60.099616105681385
-58.6178285815448
-58.335461862394475
-57.89662035013914
-57.82096885816484
-57.52580034520172
-56.79651957217022
-55.91624654655454
-55.44276794862466
-54.73432838433737
-55.12382658241074
-54.577994062759
-55.666186759844834
-55.511333678747334
-55.82361830164163
-55.82361830164163
-55.439022318501245
-55.281657100260325
-54.51134158053344
-54.11569246716037
-54.11569246716037
-53.49278111678501
-53.49278111678501
-54.04446427286451
-53.82135926850661
-54.909103596555475
-53.97362717859766
-54.909103596555475
-55.06798790405899
-54.6082578468149
-54.6082578468149
-